In [1]:
import pandas as pd
import random
from collections import defaultdict, namedtuple

In [2]:
# =========================================================================
# PARÂMETROS GERAIS
# =========================================================================
dias_semana = ["SEG","TER","QUA","QUI","SEX"]
tempos = [1,2,3,4,5]

NUM_ITERACOES = 10000  # quantas vezes tentamos alocar do zero para achar melhor solução

In [4]:
# =========================================================================
# 1) LEITURA DOS ARQUIVOS
# =========================================================================
df_prof = pd.read_excel("professores.xlsx")
df_demanda_raw = pd.read_excel("demanda.xlsx")

In [5]:
# =========================================================================
# 2) CRIAR DICIONÁRIO DE DISPONIBILIDADE
#    disponib[prof][turno][dia][tempo] = True/False
# =========================================================================
disponib = {}
for i, row in df_prof.iterrows():
    servidor = row["SERVIDOR"]
    turno    = row["TURNO"]

    if servidor not in disponib:
        disponib[servidor] = {}
    if turno not in disponib[servidor]:
        disponib[servidor][turno] = {}

    for d in dias_semana:
        if d not in disponib[servidor][turno]:
            disponib[servidor][turno][d] = {}
        for t in tempos:
            col = f"{d}{t}"  # ex.: "SEG1", "SEG2" etc.
            if col in row:
                val = row[col]
                disponib[servidor][turno][d][t] = (val == 1)
            else:
                disponib[servidor][turno][d][t] = False

In [6]:
# =========================================================================
# 3) FILTRAR "APOIO" E MONTAR DEMANDA
#    (pois "APOIO" não faz parte do quadro de horários)
# =========================================================================
df_demand = df_demanda_raw[df_demanda_raw["DISC"] != "APOIO"].copy()

Pedido = namedtuple("Pedido","prof turno nivel serie turma disc ch")
demanda = []
for i, row in df_demand.iterrows():
    demanda.append(Pedido(
        prof  = row["SERVIDOR"],
        turno = row["TURNO"],
        nivel = row["NIVEL"],
        serie = str(row["SERIE/ANO"]),
        turma = str(row["TURMA"]),
        disc  = row["DISC"],
        ch    = int(row["CH.TURMA"])
    ))

In [7]:
# =========================================================================
# 4) ESTRUTURA DE ALOCAÇÃO
#    horario[turno][nivel][dia][(serie,turma)] = { tempo: (prof, disc) }
# =========================================================================
def cria_horario_vazio(demanda):
    """Gera a estrutura vazia para todos (turno,nivel,dia,serie,turma)."""
    horario = {}
    # descobrir turnos/niveis
    combos = set()
    for ped in demanda:
        combos.add((ped.turno, ped.nivel))
    for (tnr, niv) in combos:
        if tnr not in horario:
            horario[tnr] = {}
        if niv not in horario[tnr]:
            horario[tnr][niv] = {}
        for d in dias_semana:
            horario[tnr][niv][d] = {}
    # para cada item, garantir a chave (serie,turma) em cada dia
    for ped in demanda:
        for d in dias_semana:
            if (ped.serie, ped.turma) not in horario[ped.turno][ped.nivel][d]:
                horario[ped.turno][ped.nivel][d][(ped.serie, ped.turma)] = {}
    return horario

In [8]:
# =========================================================================
# 5) FUNÇÕES DE VERIFICAÇÃO DE REGRAS
# =========================================================================

def professor_disponivel(prof, turno, dia, tempo):
    """Verifica se o professor está 'True' nesse dia/tempo."""
    if prof not in disponib:
        return False
    if turno not in disponib[prof]:
        return False
    return disponib[prof][turno][dia][tempo]

def professor_ocupado_esse_tempo(horario, prof, turno, nivel, dia, tempo):
    """Se professor já está em outra turma nesse dia/tempo."""
    for (st, dicT) in horario[turno][nivel][dia].items():
        if tempo in dicT:
            (p, dsc) = dicT[tempo]
            if p == prof:
                return True
    return False

def turma_ja_tem_aula(horario, turno, nivel, dia, serie, turma, tempo):
    """Se a turma já tem algo nesse tempo."""
    return (tempo in horario[turno][nivel][dia][(serie,turma)])

def check_buraco(horario, ped, dia):
    """
    Garante que a turma (ped.serie, ped.turma) não fique com buracos.
    i.e. se ocupa X tempos, eles devem ser 1..X sem pular, começando no 1.
    """
    dicT = horario[ped.turno][ped.nivel][dia][(ped.serie, ped.turma)]
    if not dicT:
        return False
    tempos_ocup = sorted(dicT.keys())
    if tempos_ocup[0] != 1:
        return True
    maxi = max(tempos_ocup)
    for x in range(1, maxi+1):
        if x not in dicT:
            return True
    return False

def professor_turma_ja_usou_dia(horario, ped, dia):
    """
    Nova regra: "Se CH=2, deve ser 2 dias diferentes. Se CH=3, 3 dias diferentes etc."

    Então: se já alocamos esse professor em (serie,turma) num dia, não pode alocar de novo no mesmo dia.
    """
    dicTurma = horario[ped.turno][ped.nivel][dia][(ped.serie, ped.turma)]
    for (tm, (p, dsc)) in dicTurma.items():
        if p == ped.prof:
            return True
    return False

def alocar_1tempo(horario, ped, dia, tempo):
    horario[ped.turno][ped.nivel][dia][(ped.serie, ped.turma)][tempo] = (ped.prof, ped.disc)

def desalocar_1tempo(horario, ped, dia, tempo):
    dicTurma = horario[ped.turno][ped.nivel][dia][(ped.serie, ped.turma)]
    if tempo in dicTurma:
        del dicTurma[tempo]

def pode_alocar(horario, ped, dia, tempo):
    """
    Checa se podemos alocar esse (prof,disc) no dia/tempo,
    sem violar:
      1) professor disponível
      2) professor não em outra turma nesse dia/tempo
      3) sem buraco
      4) professor não pode repetir esse dia na mesma série/turma (CH deve ser espalhado em dias distintos)
    """
    # 1) e 2)
    if not professor_disponivel(ped.prof, ped.turno, dia, tempo):
        return False
    if professor_ocupado_esse_tempo(horario, ped.prof, ped.turno, ped.nivel, dia, tempo):
        return False
    # 4) se professor_turma_ja_usou_dia => recusa
    if professor_turma_ja_usou_dia(horario, ped, dia):
        return False

    # Tenta alocar e checar buraco
    alocar_1tempo(horario, ped, dia, tempo)
    if check_buraco(horario, ped, dia):
        # se gerou buraco, desfaz
        desalocar_1tempo(horario, ped, dia, tempo)
        return False

    return True

In [9]:
# =========================================================================
# 6) HEURÍSTICA: ALOCAR A DEMANDA (SIMPLES) + ITERAÇÕES
# =========================================================================
def alocar_demanda(horario, demanda):
    """
    Aloca cada (prof,disc,CH) em dias distintos.
    Vamos varrer a 'demanda' e tentar distribuir as aulas do professor
    para essa série/turma em dias diferentes, sem pular tempos da turma.
    Retorna lista de (pedido, quant_nao_alocado).
    """
    nao_alocados = []
    # Embaralhar para randomizar
    dem_local = list(demanda)
    random.shuffle(dem_local)

    for ped in dem_local:
        ch_rest = ped.ch

        # Varremos dias em ordem aleatória
        dias_rand = list(dias_semana)
        random.shuffle(dias_rand)

        for d in dias_rand:
            if ch_rest <= 0:
                break
            # Tentar achar um tempo livre
            tempos_rand = list(tempos)
            random.shuffle(tempos_rand)
            # Se professor já usou esse dia p/ essa turma, não pode alocar => mas a checagem tá em "pode_alocar(...)"
            for tm in tempos_rand:
                if ch_rest <= 0:
                    break
                if pode_alocar(horario, ped, d, tm):
                    ch_rest -= 1

        if ch_rest > 0:
            # sobrou
            nao_alocados.append((ped, ch_rest))

    return nao_alocados

def constroi_solucao(demanda):
    """Cria horario vazio e aloca com heurística, retornando (horario, nao_alocados)."""
    horario_temp = cria_horario_vazio(demanda)
    nao_aloc = alocar_demanda(horario_temp, demanda)
    return horario_temp, nao_aloc

melhor_horario = None
melhor_nao_alocados = None
melhor_sobra = float("inf")

for it in range(NUM_ITERACOES):
    horario_cand, nao_aloc_cand = constroi_solucao(demanda)
    sobra = sum(x[1] for x in nao_aloc_cand)
    if sobra < melhor_sobra:
        melhor_sobra = sobra
        melhor_horario = horario_cand
        melhor_nao_alocados = nao_aloc_cand

# Ao final, temos a melhor alocação encontrada
if melhor_sobra == 0:
    print(f"Ótimo! Conseguimos alocar tudo (0 pendências) após {NUM_ITERACOES} iterações.")
else:
    print(f"[Aviso] Não foi possível alocar {melhor_sobra} tempos.")
    print(f"Melhor resultado após {NUM_ITERACOES} iterações.")
    print("LISTA DE NÃO ALOCADOS:")
    for (ped, c) in melhor_nao_alocados:
        if c>0:
            print(f" - {ped.prof}, {ped.disc}, {ped.turno}, {ped.nivel}, {ped.serie}-{ped.turma}, restam {c} aulas")

    # Se quiser salvar isso em excel:
    df_na = []
    for (ped, c) in melhor_nao_alocados:
        if c>0:
            df_na.append({
                "SERVIDOR": ped.prof,
                "DISCIPLINA": ped.disc,
                "TURNO": ped.turno,
                "NIVEL": ped.nivel,
                "SERIE": ped.serie,
                "TURMA": ped.turma,
                "NAO_ALOCADO": c
            })
    df_na_x = pd.DataFrame(df_na)
    df_na_x.to_excel("NAO_ALOCADOS.xlsx", index=False)

[Aviso] Não foi possível alocar 25 tempos.
Melhor resultado após 10000 iterações.
LISTA DE NÃO ALOCADOS:
 - CHRISTINA SIMAS CORREA, LTEC, VES, ENSIN, 1-5, restam 2 aulas
 - RAELESON LIMA COELHO, FISIC, VES, ENSIN, 2-6, restam 1 aulas
 - WELIGTHON JOSE MARTINS, MAT, VES, ENSIN, 2-6, restam 3 aulas
 - WELIGTHON JOSE MARTINS, MAT, VES, ENSIN, 2-7, restam 1 aulas
 - VAGA 02 , GEO, VES, ENSIN, 2-2, restam 1 aulas
 - VAGA 02 , CHSA, VES, ENSIN, 1-1, restam 1 aulas
 - AVONEIDE DA SILVA MEND, MAT, VES, ENSIN, 3-2, restam 1 aulas
 - AYRTON LUCAS LIMA TELE, CIENT, VES, ENSIN, 2-5, restam 1 aulas
 - CHRISTINA SIMAS CORREA, LININ, VES, ENSIN, 1-5, restam 1 aulas
 - WELIGTHON JOSE MARTINS, MAT, VES, ENSIN, 2-5, restam 1 aulas
 - VAGA 02 , GEO, VES, ENSIN, 1-8, restam 2 aulas
 - AVONEIDE DA SILVA MEND, MAT, VES, ENSIN, 1-7, restam 1 aulas
 - GLAURIA GLEICE GAMA DO, LPSL, VES, ENSIN, 2-7, restam 1 aulas
 - ELIZABETH DE OLIVEIRA, LPSL, VES, ENSIN, 3-5, restam 1 aulas
 - PRISCILA VIANA DE ARAU, EDFIS, 

In [10]:
# =========================================================================
# 7) GERA ARQUIVO FINAL: (A) POR TURMA, (B) POR PROFESSOR
# =========================================================================

# (A) Por Turma
def gera_df_por_turma(horario):
    from collections import defaultdict
    colunas_por_tn = defaultdict(set)

    # descobrir colunas
    for tnr in horario:
        for niv in horario[tnr]:
            for d in dias_semana:
                for (sa, tu), tempos_dic in horario[tnr][niv][d].items():
                    colunas_por_tn[(tnr,niv)].add(f"{sa}-{tu}")

    # montar
    dfs_por_aba = {}
    for tnr in horario:
        for niv in horario[tnr]:
            turmas_cols = sorted(list(colunas_por_tn[(tnr,niv)]))
            linhas = []
            for d in dias_semana:
                for tm in tempos:
                    row_dict = {
                        "TURNO": tnr,
                        "NIVEL": niv,
                        "DIA": d,
                        "TEMPO": tm
                    }
                    for col_turma in turmas_cols:
                        row_dict[col_turma] = ""
                    # preencher
                    for (sa, tuma), tdict in horario[tnr][niv][d].items():
                        if tm in tdict:
                            (p, ds) = tdict[tm]
                            colname = f"{sa}-{tuma}"
                            row_dict[colname] = f"{p}+{ds}"
                    linhas.append(row_dict)
            df_aba = pd.DataFrame(linhas)
            nome_aba = f"{tnr}_{niv}"
            dfs_por_aba[nome_aba] = df_aba
    return dfs_por_aba

# (B) Por Professor
def gera_df_por_professor(horario):
    # Precisamos mapear: professor => em qual dia/tempo => quais turmas/disciplinas
    prof_data = defaultdict(lambda: defaultdict(list))
    prof_cols = defaultdict(set)  # p => set de colunas (serie-turma(disc))

    for tnr in horario:
        for niv in horario[tnr]:
            for d in dias_semana:
                for (sa, tu), tdict in horario[tnr][niv][d].items():
                    for tm, (p, ds) in tdict.items():
                        col = f"{sa}-{tu}({ds})"
                        prof_cols[p].add(col)
                        prof_data[p][(d,tm)].append( (tnr,niv, sa, tu, ds) )

    dfs_por_prof = {}
    for p in prof_data:
        colunas = sorted(list(prof_cols[p]))
        linhas = []
        for d in dias_semana:
            for tm in tempos:
                row_dict = {
                    "PROFESSOR": p,
                    "DIA": d,
                    "TEMPO": tm
                }
                for c in colunas:
                    row_dict[c] = ""
                if (d,tm) in prof_data[p]:
                    for (tnr,niv, sa, tu, ds) in prof_data[p][(d,tm)]:
                        cname = f"{sa}-{tu}({ds})"
                        # valor: "tnr-niv" ou algo do tipo
                        row_dict[cname] = f"{tnr}-{niv}"
                linhas.append(row_dict)
        df_p = pd.DataFrame(linhas)
        dfs_por_prof[p] = df_p
    return dfs_por_prof

# Gera e salva
horario_final = melhor_horario
dfs_turma = gera_df_por_turma(horario_final)
dfs_prof  = gera_df_por_professor(horario_final)

with pd.ExcelWriter("QUADRO_HORARIOS_TURMAS.xlsx") as writer:
    for aba, df_ in dfs_turma.items():
        df_.to_excel(writer, sheet_name=aba[:31], index=False)

with pd.ExcelWriter("QUADRO_HORARIOS_PROFESSORES.xlsx") as writer:
    for p, df_ in dfs_prof.items():
        nome_aba = p[:31]  # Excel limita a 31 chars
        df_.to_excel(writer, sheet_name=nome_aba, index=False)

print("Arquivos gerados:")
print(" - QUADRO_HORARIOS_TURMAS.xlsx (uma aba por Turno_Nivel)")
print(" - QUADRO_HORARIOS_PROFESSORES.xlsx (uma aba por professor)")
print(" - NAO_ALOCADOS.xlsx (caso haja não alocações)")

Arquivos gerados:
 - QUADRO_HORARIOS_TURMAS.xlsx (uma aba por Turno_Nivel)
 - QUADRO_HORARIOS_PROFESSORES.xlsx (uma aba por professor)
 - NAO_ALOCADOS.xlsx (caso haja não alocações)
